In [1]:
import cv2
import numpy as np
import tensorflow as tf
from tensorflow import keras
import csv

In [2]:
model = keras.models.load_model('main_faces.h5')

In [3]:
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

In [4]:
label_to_name = {
    0: 'image_0',
    1: 'image_1',
    2: 'image_2',
    3: 'image_3',
    4: 'image_4',
    5: 'image_5',
    6: 'image_6',
    7: 'image_7',
    8: 'Pratik'
}

In [5]:
recognized_people = set()

In [6]:
# Start the video capture
cap = cv2.VideoCapture(0)

In [7]:
while True:
    # Read a frame from the video capture
    ret, frame = cap.read()
    
    # Convert the frame to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # Detect faces in the frame using the Haar cascade classifier
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)
    
    # Loop over the detected faces
    for (x,y,w,h) in faces:
        # Extract the face region from the frame
        face_region = frame[y:y+h, x:x+w]
        
        # Preprocess the face region for the Keras model
        face_region = cv2.resize(face_region, (150, 150))
        face_region = face_region / 255.0
        face_region = np.expand_dims(face_region, axis=0)
        
        # Make a prediction using the Keras model
        prediction = model.predict(face_region)
        
        # Get the predicted class label and name
        predicted_class = np.argmax(prediction[0])
        predicted_name = label_to_name[predicted_class]
        
        # Draw a rectangle around the face and display the predicted name
        cv2.rectangle(frame, (x,y), (x+w,y+h), (255,0,0), 2)
        cv2.putText(frame, predicted_name, (x,y-10), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,0,0), 2)
        
        # Add the predicted name to the set of recognized people
        recognized_people.add(predicted_name)
    
    # Display the frame with the detected faces and predicted names
    cv2.imshow('frame', frame)
    
    # Check if the user pressed the 'q' key
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the video capture and destroy all windows
cap.release()
cv2.destroyAllWindows()

1/1 [==============================] - 0s 22ms/step


In [8]:
with open('attendance.csv', 'w') as f:
    writer = csv.writer(f)
    
    # Write the header row
    writer.writerow(['Name', 'Attendance'])
    
    # Write a row for each person in label_to_name dictionary
    for name in label_to_name.values():
        if name in recognized_people:
            writer.writerow([name, 'present'])